In [19]:
%pylab inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import re
import fnmatch
import errno    
import json
import glob
from riboraptor.sradb import SRAdb
from riboraptor.helpers import path_leaf
from collections import defaultdict
from riboraptor.utils import summary_starlogs_over_runs

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
summary_starlogs_over_runs('/staging/as/skchoudh/re-ribo-analysis/hg38/SRP017942/starlogs/')

In [4]:
datasets = {}
datasets['hg38'] = pd.read_excel('../data/datasets/re-ribo-datasets.xlsx', sheet_name='hg38')
datasets['mm10'] = pd.read_excel('../data/datasets/re-ribo-datasets.xlsx', sheet_name='mm10')
datasets['mm10'] = pd.read_excel('../data/datasets/re-ribo-datasets.xlsx', sheet_name='mm10')


In [5]:
geodb = SRAdb('/staging/as/skchoudh/GEOmetadb.sqlite')
sradb = SRAdb('/staging/as/skchoudh/SRAmetadb.sqlite')

re_ribo_analysis_dir = '/staging/as/skchoudh/re-ribo-analysis/'

In [6]:
geodb.get_query("SELECT * FROM gse WHERE gse='GSE22004'")

,ID,contact,contributor,gse,last_update_date,overall_design,pubmed_id,repeats,repeats_sample_list,status,submission_date,summary,supplementary_file,title,type,variable,variable_description,web_link
0,19587.0,Name: Huili Guo;\tEmail: hguo@mit.edu;\tPhone:...,None,GSE22004,2013-06-07,Refer to individual Series.,20703300,None,None,Public on Aug 03 2010,2010-05-26,MicroRNAs (miRNAs) are endogenous ~22-nucleoti...,ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE22nnn...,Mammalian microRNAs predominantly act to decre...,Expression profiling by high throughput sequen...,None,None,None


In [7]:
df = pd.read_html('../data/datasets/hrpdviewer.html')

In [8]:
df = df[0].sort_values(by='Year')


In [12]:
df['SRP'] = df['GEO number'].apply(lambda x: geodb.convert_gse_to_srp(x))

In [13]:
df[df['GEO number'].str.contains('SRP')]['SRP'] = df[df['GEO number'].str.contains('SRP')]['GEO number']

/home/cmb-panasas2/skchoudh/software_frozen/anaconda27/envs/riboraptor/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [17]:
df.to_csv('../data/datasets/hrpdviewer.tsv', sep='\t', header=True, index=False)

In [20]:
[len(df['GEO number'].unique())]

68

In [24]:
datasets = defaultdict(list)

In [26]:
for directory in glob.glob('{}/*/*'.format(re_ribo_analysis_dir)):
    srp = path_leaf(directory)
    assembly = path_leaf(os.path.dirname(directory))
    datasets[srp].append(assembly)

In [27]:
datasets

defaultdict(list,
            {'SRP003554': ['mm10'],
             'SRP007567': ['mm10'],
             'SRP010679_fixed_adapters': ['hg38'],
             'SRP017942': ['hg38', 'mm10'],
             'SRP021058': ['mm10'],
             'SRP028243': ['BDGP6'],
             'SRP028243_fixed_adapters': ['BDGP6'],
             'SRP030014': ['mm10'],
             'SRP031501': ['hg38', 'mm10'],
             'SRP033366': ['BDGP6'],
             'SRP038695': ['hg38'],
             'SRP042937': ['hg38'],
             'SRP043036': ['sacCerR64'],
             'SRP045214': ['hg38'],
             'SRP045475': ['BDGP6'],
             'SRP048708': ['mm10'],
             'SRP049168': ['hg38', 'mm10'],
             'SRP055009': ['hg38'],
             'SRP055707': ['sacCerR64'],
             'SRP056576': ['mm10'],
             'SRP056647': ['sacCerR64'],
             'SRP062409': ['mm10'],
             'SRP063562': ['MG1655'],
             'SRP064304': ['mm10'],
             'SRP074765': ['mm10'],
       

In [21]:
file_name = '../data/datasets/rpfdb_metadata.json'
rpfdb_data = json.load(open(file_name))
rpfdb_df = pd.DataFrame(rpfdb_data)
rpfdb_df = rpfdb_df.drop(columns=['Abstract'])
rpfdb_df[['Author', 'Year']] = rpfdb_df.Author_year.str.split(',', n=1, expand=True)
rpfdb_df = rpfdb_df.drop(columns=['Author_year'])
rpfdb_df.Year = rpfdb_df.Year.astype(int)
species_value_counts = rpfdb_df.Species.value_counts() 
rpfdb_df.head()
rpfdb_df = rpfdb_df.sort_values(by=['Year', 'Species'])




In [24]:
rpfdb_df[rpfdb_df.Species == 'Zebrafish']

,Citation,Index,Ref,Samples,Species,Strain,Study,Title,Treatment,Year,Author
30,Ribosome profiling shows that miR-430 reduces ...,211,GRCz11,12,Zebrafish,strain: TUAB,GSE34743,Ribosome profiling of early zebrafish embryos ...,100 ug/ml Cycloheximide,2012,Bazzini AA
27,"Nanog, Pou5f1 and SoxB1 activate zygotic gene ...",210,GRCz11,4,Zebrafish,strain: TUAB,GSE47558,"Nanog, SoxB1 and Pou5f1/Oct4 regulate widespre...",50 μg/ml cycloheximide,2013,Lee MT
29,Ribosome profiling reveals resemblance between...,209,GRCz11,8,Zebrafish,strain: TL/AB,GSE46512,Ribosome Profiling over a Zebrafish Developmen...,100 μg/ml cycloheximide,2013,Chew GL
26,Identification of small ORFs in vertebrates us...,208,GRCz11,18,Zebrafish,strain: TUAB,GSE53693,Identification of small ORFs in vertebrates us...,100ug/ml Cycloheximide,2014,Bazzini AA
28,Poly(A)-tail profiling reveals an embryonic sw...,207,GRCz11,1,Zebrafish,strain: AB,GSE52809,Poly(A)-tail profiling reveals an embryonic sw...,100 µg/ml cycloheximide,2014,Subtelny AO
